ABAG Landuse Calculations for VTA and BART Model.

This notebook documents the joining data between VTA or CCAG TAZ Zones and MTC's 1454 regions.  VTA and CCAG will be reffered to as Regional Partners. 

Also there is details on how to convert ABAG numbers to BART numbers.

The inputs are the yearly calculations from MTC and VTA and CCAG.  The outputs are a csv, excel, and a shapefile.

In [24]:
#This uses geopy36
import geopandas as gpd
import pandas as pd
%matplotlib inline  
from shapely.geometry import Point

from simpledbf import Dbf5
#pip install simpledbf
#https://pypi.python.org/pypi/simpledbf/0.2.4

In [25]:
# RP stands for Regional Partner
def prep_data(abag, rp_taz, geom_given = True):
    
    cols = ['TAZ','TAZ1454','DIST','SDIST','COUNTY','TOTHH','TOTPOP',
                                    'HHPOP','EMPRES','HH1','HH2','HH3','HH4','TACRES','RESACRE','CIACRE','TEMP',
                                    'RETEMP','SEREMP','OTHEMP','AGEMP','MANEMP','WHOEMP','AGE0004','AGE0519',
                                    'AGE2044','AGE4564','AGE65','AGE0513','AGE1417','AGE1824','SFHH','MFHH', 'CITY']
    cols_to_retain = ['INC1','INC2','INC3','INC4','MHHINC', 'ESENR', 'HSENR', 'COLLENR', 'COLLENRF', 'COLLENRP']

    cols.extend(cols_to_retain)

    if geom_given:
        cols.append('geometry')
    else:
        pass
    
    
    rp_taz = rp_taz[cols].rename(columns={"TOTHH":"RP_TOTHH","TOTPOP":"RP_TOTPOP",
                                "HHPOP":"RP_HHPOP","EMPRES":"RP_EMPRES","RESACRE":'RP_RESACRE',"CIACRE":"RP_CIACRE","TEMP":"RP_TEMP",
                                         "AGE0004":"RP_AGE0004","AGE0519":"RP_AGE0519","AGE2044":"RP_AGE2044","AGE4564":"RP_AGE4564","AGE65":"RP_AGE65"})
    #Clear out Student calc numbers where TOTPOP is 0.
    criteria = "RP_TOTPOP==0&(AGE0513>0|AGE1417>0|AGE1824>0)"
    rp_taz.loc[rp_taz.eval(criteria), 'AGE0513'] = 0
    rp_taz.loc[rp_taz.eval(criteria), 'AGE1417'] = 0
    rp_taz.loc[rp_taz.eval(criteria), 'AGE1824'] = 0
    
    #Calculation district wide average number of students to population ratio
    okay_criteria = "RP_TOTPOP>AGE0513+AGE1417+AGE1824"
    AGE0513_avg_factor =  (rp_taz.query(okay_criteria).groupby('DIST')['AGE0513'].sum()/rp_taz.query(okay_criteria).groupby('DIST')['RP_TOTPOP'].sum()).reset_index().rename(columns={0:'AGE0513_factor'})
    AGE1417_avg_factor =  (rp_taz.query(okay_criteria).groupby('DIST')['AGE1417'].sum()/rp_taz.query(okay_criteria).groupby('DIST')['RP_TOTPOP'].sum()).reset_index().rename(columns={0:'AGE1417_factor'})    
    AGE1824_avg_factor =  (rp_taz.query(okay_criteria).groupby('DIST')['AGE1824'].sum()/rp_taz.query(okay_criteria).groupby('DIST')['RP_TOTPOP'].sum()).reset_index().rename(columns={0:'AGE1824_factor'})
    rp_taz = pd.merge(pd.merge(pd.merge(rp_taz, AGE0513_avg_factor),AGE1417_avg_factor),AGE1824_avg_factor)
    
    
    # Save percentages for later calc
    rp_taz['AGE0513_percent'] = rp_taz['AGE0513']/rp_taz['RP_TOTPOP']
    rp_taz['AGE1417_percent'] = rp_taz['AGE1417']/rp_taz['RP_TOTPOP']
    rp_taz['AGE1824_percent'] = rp_taz['AGE1824']/rp_taz['RP_TOTPOP']

    #If the Total Population for the zone is below the student population, overwrite percentages with district factors.
    bad_criteria = "RP_TOTPOP<AGE0513+AGE1417+AGE1824"
    rp_taz.loc[rp_taz.eval(bad_criteria), 'AGE0513_percent'] = rp_taz.loc[rp_taz.eval(bad_criteria), 'AGE0513_factor']
    rp_taz.loc[rp_taz.eval(bad_criteria), 'AGE1417_percent'] = rp_taz.loc[rp_taz.eval(bad_criteria), 'AGE1417_factor']
    rp_taz.loc[rp_taz.eval(bad_criteria), 'AGE1824_percent'] = rp_taz.loc[rp_taz.eval(bad_criteria), 'AGE1824_factor']    
    
    #If the total population is greater than ten but the student population is 0, replace student percent with district average.
    age_0513_bad_criteria = "RP_TOTPOP>=10 & AGE0513==0"
    rp_taz.loc[rp_taz.eval(age_0513_bad_criteria), 'AGE0513_percent'] = rp_taz.loc[rp_taz.eval(age_0513_bad_criteria), 'AGE0513_factor']
    age_1417_bad_criteria = "RP_TOTPOP>=10 & AGE1417==0"
    rp_taz.loc[rp_taz.eval(age_1417_bad_criteria), 'AGE1417_percent'] = rp_taz.loc[rp_taz.eval(age_1417_bad_criteria), 'AGE1417_factor']
    age_1824_bad_criteria = "RP_TOTPOP>=10 & AGE1824==0"
    rp_taz.loc[rp_taz.eval(age_1824_bad_criteria), 'AGE1824_percent'] = rp_taz.loc[rp_taz.eval(age_1824_bad_criteria), 'AGE1824_factor']    
    
    #Join the RP shapefile to the abag 2010 dataset!  Pull this source input data directly from ABAG
    rp_calc = pd.merge(abag[['TAZ1454','TOTHH','HHPOP','TOTPOP','EMPRES','HHINCQ1','HHINCQ2','HHINCQ3','HHINCQ4','RESACRE','CIACRE','TOTEMP','SHPOP62P','AGE0004','AGE0519',
                                    'AGE2044','AGE4564','AGE65P']].rename(columns={"SHPOP62P":"Z2SHARE"}),
                                rp_taz)

    return rp_calc
    # vt_calc.groupby(['TAZ1454','TAZ'])['VTA_TOTPOP'].apply(lambda x: x / x.sum())
    # vt_calc.groupby(['TAZ1454','TAZ'])['VTA_TOTPOP'].sum()

In [29]:
def employment_cleaning(rp_calc):
    sdist_totemp_sum = (rp_calc.groupby('SDIST')['RETEMP'].sum() + rp_calc.groupby('SDIST')['SEREMP'].sum() + rp_calc.groupby('SDIST')['OTHEMP'].sum() + rp_calc.groupby('SDIST')['AGEMP'].sum() + rp_calc.groupby('SDIST')['MANEMP'].sum() + rp_calc.groupby('SDIST')['WHOEMP'].sum())
    a = (rp_calc.groupby('SDIST')['RETEMP'].sum()/sdist_totemp_sum).reset_index().rename(columns={0:'RETEMP_SDIST_AVG'})
    b = (rp_calc.groupby('SDIST')['SEREMP'].sum()/sdist_totemp_sum).reset_index().rename(columns={0:'SEREMP_SDIST_AVG'})
    c = (rp_calc.groupby('SDIST')['OTHEMP'].sum()/sdist_totemp_sum).reset_index().rename(columns={0:'OTHEMP_SDIST_AVG'})
    d = (rp_calc.groupby('SDIST')['AGEMP'].sum()/sdist_totemp_sum).reset_index().rename(columns={0:'AGEMP_SDIST_AVG'})
    e = (rp_calc.groupby('SDIST')['MANEMP'].sum()/sdist_totemp_sum).reset_index().rename(columns={0:'MANEMP_SDIST_AVG'})
    f = (rp_calc.groupby('SDIST')['WHOEMP'].sum()/sdist_totemp_sum).reset_index().rename(columns={0:'WHOEMP_SDIST_AVG'})
    dfs = [a,b,c,d,e,f]
    from functools import reduce
    EMP_AVG = reduce(lambda left,right: pd.merge(left,right), dfs)
    rp_calc = pd.merge(rp_calc,EMP_AVG, how = 'left')    
    bad_criteria = 'RP_TEMP >=5 & RETEMP + SEREMP + OTHEMP + AGEMP + MANEMP + WHOEMP == 0'
    bad_criteria_spots = rp_calc.eval(bad_criteria)
    rp_calc.loc[bad_criteria_spots,'RETEMP'] = rp_calc.loc[bad_criteria_spots,'RETEMP_SDIST_AVG']
    rp_calc.loc[bad_criteria_spots,'SEREMP'] = rp_calc.loc[bad_criteria_spots,'SEREMP_SDIST_AVG']
    rp_calc.loc[bad_criteria_spots,'OTHEMP'] = rp_calc.loc[bad_criteria_spots,'OTHEMP_SDIST_AVG']
    rp_calc.loc[bad_criteria_spots,'RETEMP'] = rp_calc.loc[bad_criteria_spots,'RETEMP_SDIST_AVG']
    rp_calc.loc[bad_criteria_spots,'RETEMP'] = rp_calc.loc[bad_criteria_spots,'RETEMP_SDIST_AVG']
    rp_calc.loc[bad_criteria_spots,'RETEMP'] = rp_calc.loc[bad_criteria_spots,'RETEMP_SDIST_AVG']
    return rp_calc

In [27]:
def landuse_calcs(rp_calc):
    """ Takes the input dataframe and does transformations, share calculations, to derive numbers for projections.
    """
    rp_tothh = rp_calc.groupby(['TAZ1454','TAZ'])['RP_TOTHH'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
    rp_hhpop = rp_calc.groupby(['TAZ1454','TAZ'])['RP_HHPOP'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
    rp_totpop = rp_calc.groupby(['TAZ1454','TAZ'])['RP_TOTPOP'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
    rp_empres = rp_calc.groupby(['TAZ1454','TAZ'])['RP_EMPRES'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()

    rp_resacre = rp_calc.groupby(['TAZ1454','TAZ'])['RP_RESACRE'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
    rp_ciacre = rp_calc.groupby(['TAZ1454','TAZ'])['RP_CIACRE'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()
    rp_temp = rp_calc.groupby(['TAZ1454','TAZ'])['RP_TEMP'].sum().groupby(level = 0).transform(lambda x: x/x.sum()).reset_index()

    rp_tothh = rp_tothh.rename(columns={"RP_TOTHH":"RP_TOTHH_share"})
    rp_hhpop = rp_hhpop.rename(columns={"RP_HHPOP":"RP_HHPOP_share"})
    rp_totpop = rp_totpop.rename(columns={"RP_TOTPOP":"RP_TOTPOP_share"})
    rp_empres = rp_empres.rename(columns={"RP_EMPRES":"RP_EMPRES_share"})

    rp_resacre = rp_resacre.rename(columns={"RP_RESACRE":"RP_RESACRE_share"})
    rp_ciacre = rp_ciacre.rename(columns={"RP_CIACRE":"RP_CIACRE_share"})
    rp_temp = rp_temp.rename(columns={"RP_TEMP":"RP_TEMP_share"})

    vta_final= pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(rp_calc,rp_tothh),rp_hhpop),rp_totpop),rp_empres),rp_resacre),rp_ciacre),rp_temp)
    vta_final['abag_TOTHH_dist'] = vta_final['TOTHH']*vta_final['RP_TOTHH_share']
    vta_final['abag_HHPOP_dist'] = vta_final['HHPOP']*vta_final['RP_HHPOP_share']
    vta_final['abag_TOTPOP_dist'] = vta_final['TOTPOP']*vta_final['RP_TOTPOP_share']
    vta_final['abag_EMPRES_dist'] = vta_final['EMPRES']*vta_final['RP_EMPRES_share']
    
    vta_final['abag_RESACRE_dist'] = round(vta_final['RESACRE']*vta_final['RP_RESACRE_share'])
    vta_final['abag_CIACRE_dist'] = round(vta_final['CIACRE']*vta_final['RP_CIACRE_share'])
    vta_final['abag_TEMP_dist'] = round(vta_final['TOTEMP']*vta_final['RP_TEMP_share'])

#     vta_final['abag_HHPOP_dist'] = round(vta_final['RP_HHPOP']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])
    
#     vta_final['abag_TOTPOP_dist'] = round(vta_final['RP_TOTPOP']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])

#     vta_final['abag_EMPRES_dist'] = round(vta_final['RP_EMPRES']/vta_final['RP_TOTHH']*vta_final['abag_TOTHH_dist'])

    #Make HH
    vta_final['abag_HH1_dist'] =round(vta_final['HH1']/(vta_final['HH1'] + vta_final['HH2'] + vta_final['HH3'] + vta_final['HH4'])*vta_final['abag_TOTHH_dist'])
    vta_final['abag_HH2_dist'] =round(vta_final['HH2']/(vta_final['HH1'] + vta_final['HH2'] + vta_final['HH3'] + vta_final['HH4'])*vta_final['abag_TOTHH_dist'])
    vta_final['abag_HH3_dist'] =round(vta_final['HH3']/(vta_final['HH1'] + vta_final['HH2'] + vta_final['HH3'] + vta_final['HH4'])*vta_final['abag_TOTHH_dist'])
    vta_final['abag_HH4_dist'] =round(vta_final['HH4']/(vta_final['HH1'] + vta_final['HH2'] + vta_final['HH3'] + vta_final['HH4'])*vta_final['abag_TOTHH_dist'])

#     vta_final['abag_RETEMP_dist'] = round(vta_final['RETEMP']/vta_final['RP_TEMP']*vta_final['abag_TEMP_dist'])
    vta_final['abag_RETEMP_dist'] = round(vta_final['RETEMP']/(vta_final['RETEMP'] + vta_final['SEREMP'] + vta_final['OTHEMP'] + vta_final['AGEMP'] + vta_final['MANEMP'] + vta_final['WHOEMP'])*vta_final['abag_TEMP_dist'])
    vta_final['abag_SEREMP_dist'] = round(vta_final['SEREMP']/(vta_final['RETEMP'] + vta_final['SEREMP'] + vta_final['OTHEMP'] + vta_final['AGEMP'] + vta_final['MANEMP'] + vta_final['WHOEMP'])*vta_final['abag_TEMP_dist'])
    vta_final['abag_OTHEMP_dist'] = round(vta_final['OTHEMP']/(vta_final['RETEMP'] + vta_final['SEREMP'] + vta_final['OTHEMP'] + vta_final['AGEMP'] + vta_final['MANEMP'] + vta_final['WHOEMP'])*vta_final['abag_TEMP_dist'])
    vta_final['abag_AGEMP_dist'] = round(vta_final['AGEMP']/(vta_final['RETEMP'] + vta_final['SEREMP'] + vta_final['OTHEMP'] + vta_final['AGEMP'] + vta_final['MANEMP'] + vta_final['WHOEMP'])*vta_final['abag_TEMP_dist'])
    vta_final['abag_MANEMP_dist'] = round(vta_final['MANEMP']/(vta_final['RETEMP'] + vta_final['SEREMP'] + vta_final['OTHEMP'] + vta_final['AGEMP'] + vta_final['MANEMP'] + vta_final['WHOEMP'])*vta_final['abag_TEMP_dist'])
    vta_final['abag_WHOEMP_dist'] = round(vta_final['WHOEMP']/(vta_final['RETEMP'] + vta_final['SEREMP'] + vta_final['OTHEMP'] + vta_final['AGEMP'] + vta_final['MANEMP'] + vta_final['WHOEMP'])*vta_final['abag_TEMP_dist'])

    vta_final['abag_AGE0004_dist'] = round(vta_final['RP_AGE0004']/(vta_final['RP_AGE0004'] + vta_final['RP_AGE0519'] + vta_final['RP_AGE2044'] + vta_final['RP_AGE4564'] + vta_final['RP_AGE65'])*vta_final['abag_TOTPOP_dist'])
    vta_final['abag_AGE0519_dist'] = round(vta_final['RP_AGE0519']/(vta_final['RP_AGE0004'] + vta_final['RP_AGE0519'] + vta_final['RP_AGE2044'] + vta_final['RP_AGE4564'] + vta_final['RP_AGE65'])*vta_final['abag_TOTPOP_dist'])
    vta_final['abag_AGE2044_dist'] = round(vta_final['RP_AGE2044']/(vta_final['RP_AGE0004'] + vta_final['RP_AGE0519'] + vta_final['RP_AGE2044'] + vta_final['RP_AGE4564'] + vta_final['RP_AGE65'])*vta_final['abag_TOTPOP_dist'])
    vta_final['abag_AGE4564_dist'] = round(vta_final['RP_AGE4564']/(vta_final['RP_AGE0004'] + vta_final['RP_AGE0519'] + vta_final['RP_AGE2044'] + vta_final['RP_AGE4564'] + vta_final['RP_AGE65'])*vta_final['abag_TOTPOP_dist'])
    vta_final['abag_AGE65_dist'] = round(vta_final['RP_AGE65']/(vta_final['RP_AGE0004'] + vta_final['RP_AGE0519'] + vta_final['RP_AGE2044'] + vta_final['RP_AGE4564'] + vta_final['RP_AGE65'])*vta_final['abag_TOTPOP_dist'])

    vta_final['abag_AGE0513_dist'] = round(vta_final['AGE0513_percent']*vta_final['abag_TOTPOP_dist'])
    vta_final['abag_AGE1417_dist'] = round(vta_final['AGE1417_percent']*vta_final['abag_TOTPOP_dist'])
    vta_final['abag_AGE1824_dist'] = round(vta_final['AGE1824_percent']*vta_final['abag_TOTPOP_dist'])
    
    vta_final['abag_SFHH_dist'] = round(vta_final['SFHH']/(vta_final['SFHH'] + vta_final['MFHH'])*vta_final['abag_TOTHH_dist'])
    vta_final['abag_MFHH_dist'] = round(vta_final['MFHH']/(vta_final['SFHH'] + vta_final['MFHH'])*vta_final['abag_TOTHH_dist'])

    bad_criteria = "abag_HHPOP_dist > abag_TOTPOP_dist"
    vta_final.loc[vta_final.eval(bad_criteria), 'abag_HHPOP_dist'] = vta_final.loc[vta_final.eval(bad_criteria), 'abag_TOTPOP_dist']

    bad_criteria = "abag_EMPRES_dist > abag_HHPOP_dist"
    vta_final.loc[vta_final.eval(bad_criteria), 'abag_EMPRES_dist'] = vta_final.loc[vta_final.eval(bad_criteria), 'abag_HHPOP_dist']
    
    bad_criteria = "abag_TOTHH_dist > 0 & abag_HHPOP_dist == 0 & TAZ <= 1490"
    city_hhpop_tothh_average = (vta_final.groupby(["CITY"])['abag_HHPOP_dist'].sum() / vta_final.groupby(["CITY"])['abag_TOTHH_dist'].sum()).reset_index().rename(columns={0:'HHPOP/TOTHH'})
    vta_final = pd.merge(vta_final, city_hhpop_tothh_average, how = 'left')
    vta_final.loc[vta_final.eval(bad_criteria), 'abag_TOTPOP_dist'] = vta_final.loc[vta_final.eval(bad_criteria), 'abag_TOTHH_dist']*vta_final.loc[vta_final.eval(bad_criteria), 'HHPOP/TOTHH']
    vta_final.loc[vta_final.eval(bad_criteria), 'abag_HHPOP_dist'] = vta_final.loc[vta_final.eval(bad_criteria), 'abag_TOTHH_dist']*vta_final.loc[vta_final.eval(bad_criteria), 'HHPOP/TOTHH']

    bad_criteria = "abag_TOTHH_dist > 0 & abag_HHPOP_dist == 0 & TAZ > 1490"    
    sdist_hhpop_tothh_average = (vta_final.groupby(["SDIST"])['abag_HHPOP_dist'].sum() / vta_final.groupby(["SDIST"])['abag_TOTHH_dist'].sum()).reset_index().rename(columns={0:'SD_HHPOP/TOTHH'})
    vta_final = pd.merge(vta_final, sdist_hhpop_tothh_average, how = 'left')
    vta_final.loc[vta_final.eval(bad_criteria), 'abag_TOTPOP_dist'] = vta_final.loc[vta_final.eval(bad_criteria), 'abag_TOTHH_dist']*vta_final.loc[vta_final.eval(bad_criteria), 'SD_HHPOP/TOTHH']
    vta_final.loc[vta_final.eval(bad_criteria), 'abag_HHPOP_dist'] = vta_final.loc[vta_final.eval(bad_criteria), 'abag_TOTHH_dist']*vta_final.loc[vta_final.eval(bad_criteria), 'SD_HHPOP/TOTHH']
    
    vta_final = vta_final.round({'abag_TOTHH_dist': 0, 'abag_HHPOP_dist': 0, 'abag_TOTPOP_dist': 0, 'abag_EMPRES_dist': 0})
    
    return vta_final

In [13]:
# Prepping the MTC data
mtc_taz = gpd.read_file('MTC/')
abag = pd.read_csv('ABAG_03202018/run7224c_taz_summaries_2015.csv')
abag = abag.rename(columns={'ZONE':'TAZ1454'})
mtc_taz = pd.merge(mtc_taz,abag)

mtc_taz.head()

,STFID,FIPSSTCO,TRACT2,TRACT,TRACTID,SUPERD,TAZ1454,AREALAND,AREAWATR,LANDACRE,...,HHPOP,TOTPOP,EMPRES,AGE0004,AGE0519,AGE2044,AGE4564,AGE65P,total_job_spaces,total_residential_units
0,06001400100,06001,400100,400100,4001,19,1005,6799198,0,1680.118487,...,3027.0,3029.0,2086.0,119.0,249.0,799.0,1312.0,550.0,3344.0,1399.0
1,06001400200,06001,400200,400200,4002,19,999,659615,0,162.994423,...,1928.0,2002.0,1403.0,92.0,179.0,833.0,674.0,224.0,6134.0,909.0
2,06001400300,06001,400300,400300,4003,19,998,1074640,0,265.549338,...,5284.0,5325.0,3906.0,282.0,444.0,2592.0,1473.0,534.0,2087.0,2754.0
3,06001400400,06001,400400,400400,4004,19,1000,696057,0,171.999438,...,3711.0,3746.0,2609.0,175.0,337.0,1918.0,977.0,339.0,1434.0,1862.0
4,06001400500,06001,400500,400500,4005,19,1001,576343,0,142.417463,...,3423.0,3556.0,2280.0,171.0,430.0,1737.0,815.0,403.0,898.0,1662.0


In [ ]:
#HHINCQ1 - number of houses in the 1st quartile.
mtc_taz.columns

In [ ]:
mtc_taz.columns

In [ ]:
# Prep the VTA data

vta_taz = gpd.read_file('VTA_TAZ/')
# vta_taz = vta_taz.rename(columns={'TAZ':'VTA_TAZ'})

dbf = Dbf5('2017ABAGLanduseAllocation/VTA/zmast13.dbf')
vta_dbf = dbf.to_dataframe()
vta_dbf = vta_dbf.rename(columns={'ZONE':'TAZ'})
vta_taz = pd.merge(vta_taz,vta_dbf)


# mtc_taz = gpd.read_file('/Users/vivek/Github/VTA/Landuse/MTC/MTCTAZ1454.dbf')


#http://analytics.mtc.ca.gov/foswiki/UrbanSimTwo/OutputToTravelModel

# vta_taz['centroid'] = vta_taz.centroid
# vta_taz = vta_taz.set_geometry('centroid')
# vta_taz['old_geometry'] = vta_taz['geometry']
# vta_taz['geometry'] = vta_taz['centroid']

# centroid = gpd.sjoin(mtc_taz[['TAZ1454','geometry']], vta_taz, how = "right", op='contains')
# centroid['TAZ1454'].nunique()
# centroid[['TAZ1454','TAZ']].to_csv("rel.csv")
# rel = centroid[['TAZ1454','TAZ']]
# missing_zones = pd.DataFrame([{'TAZ1454' : 1454, 'TAZ' : 2786},{'TAZ1454':404,'TAZ':980},{'TAZ1454' : 190, 'TAZ' : 1890}])
# rel = pd.concat([rel,missing_zones])
# rel.to_csv('rel_vta_mtc.csv',index=False)

rel = pd.read_csv('rel_vta_mtc.csv')
rel = rel.loc[~rel['TAZ1454'].isnull(),]

vta_taz = pd.merge(vta_taz[['TAZ','DIST','SDIST','COUNTY','geometry','TOTHH','TOTPOP',
                            'HHPOP','EMPRES','HH1','HH2','HH3','HH4','TACRES','RESACRE','CIACRE','TEMP',
                            'RETEMP','SEREMP','OTHEMP','AGEMP','MANEMP','WHOEMP','AGE0004','AGE0519',
                            'AGE2044','AGE4564','AGE65','SFHH','MFHH']],rel)

In [30]:
# Prep Data for CCAG

# CCAG_taz = gpd.read_file('CCAGTAZ/')

# Not sure what the data in the TAZ shapefile relates to, will delete.
# del CCAG_taz['ESENR']
# del CCAG_taz['HSENR']
# CCAG_taz.rename(columns={'CITY':'CITY_NAME'}, inplace=True)
years = ['2015','2020','2025','2030','2035','2040']

dbf = Dbf5('CCAG/Zmast15.dbf')
ccag_dbf = dbf.to_dataframe()
ccag_extra_counties = ccag_dbf.query("COUNTY==10|COUNTY==11|COUNTY==12|COUNTY==13")
ccag_dbf.rename(columns={'ZONE':'TAZ'}, inplace=True)

ccag_correspondence = pd.read_excel('CCAG/CCAGTAZ-to-MTC1454-for-Vivek.xlsx', sheet_name='CCAGTAZ-MTC1454-Corresp')
ccag_correspondence.rename(columns={'CCAGTAZ':'TAZ','MTCTAZ1454':'TAZ1454'}, inplace=True)
ccag_df = pd.merge(ccag_dbf, ccag_correspondence[['TAZ','TAZ1454']], left_on = ['TAZ'], right_on = ['TAZ'])

sc_city_corr = pd.read_excel('SSC_TAZ_to_City_Correspondence.xlsx')
sc_city_corr.rename(columns={'BART_ZONE':'TAZ'}, inplace=True)
ccag_df = pd.merge(ccag_df,sc_city_corr, how = 'left')


for year in years:

    abag = pd.read_csv('ABAG_03202018/run7224c_taz_summaries_' + year + '.csv')
    abag = abag.rename(columns={'ZONE':'TAZ1454'})
    #     mtc_taz = pd.merge(mtc_taz,abag)

    rp_calc = prep_data(abag, ccag_df, geom_given = False)

    pairs = [('RP_TOTHH','TOTHH'),('RP_HHPOP','HHPOP'),('RP_TOTPOP','TOTPOP'),
             ('RP_EMPRES','EMPRES'),
             ('RP_RESACRE','RESACRE'),('RP_CIACRE','CIACRE'),('RP_TEMP','TOTEMP')]
    for pair in pairs:
        rp_calc.loc[rp_calc.query("COUNTY!=3&COUNTY!=2").index,pair[0]] = rp_calc.loc[rp_calc.query("COUNTY!=3&COUNTY!=2").index,pair[1]]

    pairs = [('HH1','HHINCQ1'),('HH2','HHINCQ2'),('HH3','HHINCQ3'),('HH4','HHINCQ4'),
             ('RP_AGE0004','AGE0004'),('RP_AGE0519','AGE0519'),('RP_AGE2044','AGE2044'),('RP_AGE4564','AGE4564'),('RP_AGE65','AGE65P')
            ]
    for pair in pairs:
        rp_calc.loc[rp_calc.index,pair[0]] = rp_calc.loc[rp_calc.index,pair[1]]
        
    rp_calc = employment_cleaning(rp_calc)        
        
    ccag_calcs = landuse_calcs(rp_calc)
    ccag_calcs = ccag_calcs.sort_values(by='TAZ')

    ccag_calcs.to_csv('output/ccag_proj_abag_calcs_' + year + '.csv',index=False)

    ccag_calcs_final = ccag_calcs[['TAZ', 'DIST', 'SDIST', 'COUNTY', 'abag_TOTHH_dist', 'abag_HHPOP_dist',
           'abag_TOTPOP_dist', 'abag_EMPRES_dist', 'abag_SFHH_dist', 'abag_MFHH_dist', 'abag_HH1_dist', 'abag_HH2_dist', 'abag_HH3_dist', 'abag_HH4_dist',
           'INC1', 'INC2', 'INC3', 'INC4', 'MHHINC', 'TACRES', 'abag_RESACRE_dist',
           'abag_CIACRE_dist', 'Z2SHARE', 'abag_TEMP_dist', 'abag_RETEMP_dist', 'abag_SEREMP_dist', 'abag_OTHEMP_dist', 'abag_AGEMP_dist',
           'abag_MANEMP_dist', 'abag_WHOEMP_dist', 'abag_AGE0004_dist', 'abag_AGE0519_dist', 'abag_AGE2044_dist', 'abag_AGE4564_dist',
           'abag_AGE65_dist', 'abag_AGE0513_dist', 'abag_AGE1417_dist', 'abag_AGE1824_dist', 'ESENR', 'HSENR',
           'COLLENR', 'COLLENRF', 'COLLENRP','TAZ1454']].rename(columns={'TAZ':'ZONE'})

    ccag_calcs_final.rename(columns=lambda x: x.replace('abag_','').replace('_dist',''),inplace=True)

    del ccag_calcs_final['TAZ1454']
    col_order = ccag_calcs_final.columns
    ccag_calcs_final_appended = ccag_calcs_final.append(ccag_extra_counties)[col_order]

    ccag_calcs_final_appended.to_csv('output/ccag_calcs_clean' + year + '.csv',na_rep=0,index=False)

In [31]:
#Prep ABAG data
#BART: Update abag projection to new file.

# mtc_taz = gpd.read_file('MTC/')
# mtc_taz = mtc_taz.rename(columns={'TAZ1454':'ZONE'})
years = ['2015','2020','2025','2030','2035','2040']

#Prep BART data

# bart_dbf = pd.read_excel('bart_calcs_clean2015_AvdH_corrected.xlsx', sheet_name='Corrected')
# year = '2015'

# This might be inefficient since the relationship seems to be inside the bart_TAZ shapefile.
# bart_taz = gpd.read_file('BART-TAZ/TAZ/')
# bart_crs = bart_taz.crs
# bart_taz = pd.merge(bart_taz[['TAZ','KEY','AREA', 'geometry']],bart_dbf)

for year in years:

    if(year in ['2015','2020','2025','2030']):
        print(year)
        bart_dbf = pd.read_csv('bart_calcs_clean2015_AvdH_input.csv')
        bart_extra_counties = bart_dbf.query("COUNTY==10|COUNTY==11|COUNTY==12|COUNTY==13")
        bart_extra_counties.rename(columns={'TAZ':'ZONE'},inplace=True)

        bart_dbf = bart_dbf.rename(columns={'ZONE':'TAZ'})
        bart_dbf['EMPRES'] = bart_dbf['EMPRES'].replace(to_replace={'#REF!':'1'}).astype(int)

    elif(year in ['2035','2040']):
        print(year)
        bart_dbf = pd.read_excel('BART_2025_2035_clean_landuse_data.xlsx', sheet_name='ZMAST35')
        bart_extra_counties = bart_dbf.query("COUNTY==10|COUNTY==11|COUNTY==12|COUNTY==13")
        bart_extra_counties.rename(columns={'TAZ':'ZONE'},inplace=True)

        bart_dbf = bart_dbf.rename(columns={'ZONE':'TAZ'})
        bart_rel = pd.read_excel('BART_zmast15_EIR.xlsx', sheet_name='Zone Corresp')
        bart_rel = bart_rel.rename(columns={'BARTZONE':'TAZ', 'MTCTAZ':'TAZ1454'})
        bart_dbf = pd.merge(bart_dbf, bart_rel)

    sc_city_corr = pd.read_excel('SSC_TAZ_to_City_Correspondence.xlsx')
    sc_city_corr.rename(columns={'BART_ZONE':'TAZ'}, inplace=True)
    bart_dbf = pd.merge(bart_dbf,sc_city_corr, how = 'left')
    
    abag = pd.read_csv('ABAG_03202018/run7224c_taz_summaries_' + year + '.csv')
    abag = abag.rename(columns={'ZONE':'TAZ1454'})
    #     mtc_taz = pd.merge(mtc_taz,abag)

    rp_calc = prep_data(abag, bart_dbf, geom_given = False)

    # Copy over values for the area that is not in the SC county or chosen region to cover up errors in the input files.
    pairs = [('RP_TOTHH','TOTHH'),('RP_HHPOP','HHPOP'),('RP_TOTPOP','TOTPOP'),
             ('RP_EMPRES','EMPRES'),
             ('RP_RESACRE','RESACRE'),('RP_CIACRE','CIACRE'),('RP_TEMP','TOTEMP')]
    for pair in pairs:
        rp_calc.loc[rp_calc.query("COUNTY!=3").index,pair[0]] = rp_calc.loc[rp_calc.query("COUNTY!=3").index,pair[1]]

    pairs = [('HH1','HHINCQ1'),('HH2','HHINCQ2'),('HH3','HHINCQ3'),('HH4','HHINCQ4'),
             ('RP_AGE0004','AGE0004'),('RP_AGE0519','AGE0519'),('RP_AGE2044','AGE2044'),('RP_AGE4564','AGE4564'),('RP_AGE65','AGE65P')
            ]
    for pair in pairs:
        rp_calc.loc[rp_calc.index,pair[0]] = rp_calc.loc[rp_calc.index,pair[1]]

    rp_calc = employment_cleaning(rp_calc)        
       
    bart_calcs = landuse_calcs(rp_calc)
    bart_calcs = bart_calcs.sort_values(by='TAZ')
    # del bart_calcs['geometry']
    bart_calcs.to_csv('output/bart_proj_abag_calcs_' + year + '.csv',index=False)
#     bart_calcs.to_excel('output/bart_proj_abag_calcs_'  + year + '.xlsx' ,index=False)

    bart_calcs_final = bart_calcs[['TAZ', 'DIST', 'SDIST', 'COUNTY', 'abag_TOTHH_dist', 'abag_HHPOP_dist',
           'abag_TOTPOP_dist', 'abag_EMPRES_dist', 'abag_SFHH_dist', 'abag_MFHH_dist', 'abag_HH1_dist', 'abag_HH2_dist', 'abag_HH3_dist', 'abag_HH4_dist',
           'INC1', 'INC2', 'INC3', 'INC4', 'MHHINC', 'TACRES', 'abag_RESACRE_dist',
           'abag_CIACRE_dist', 'Z2SHARE', 'abag_TEMP_dist', 'abag_RETEMP_dist', 'abag_SEREMP_dist', 'abag_OTHEMP_dist', 'abag_AGEMP_dist',
           'abag_MANEMP_dist', 'abag_WHOEMP_dist', 'abag_AGE0004_dist', 'abag_AGE0519_dist', 'abag_AGE2044_dist', 'abag_AGE4564_dist',
           'abag_AGE65_dist', 'abag_AGE0513_dist', 'abag_AGE1417_dist', 'abag_AGE1824_dist', 'ESENR', 'HSENR',
           'COLLENR', 'COLLENRF', 'COLLENRP','TAZ1454']].rename(columns={'TAZ':'ZONE'})

    # https://www.dataquest.io/blog/pandas-cheat-sheet/
    bart_calcs_final.rename(columns=lambda x: x.replace('abag_','').replace('_dist',''),inplace=True)

    del bart_calcs_final['TAZ1454']
    col_order = bart_calcs_final.columns
    bart_calcs_appended = bart_calcs_final.append(bart_extra_counties)[col_order]
    
    bart_calcs_appended.to_csv('output/bart_calcs_clean' + year + '.csv',na_rep=0,index=False)
#     bart_calcs_final.to_excel('output/bart_calcs_clean'  + year + '.xlsx' ,index=False)

2015


/Users/vivek/anaconda3/envs/GIS/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


2020
2025
2030
2035
2040


In [ ]:
# Generate CCAG Calculations

rp_calc = prep_data(abag, ccag_merged, rel)
vta_final = landuse_calcs(rp_calc)

geometry = vta_final['geometry']
crs = mtc_taz.crs
geo_df = gpd.GeoDataFrame(vta_final, crs=crs, geometry=geometry)

geo_df.head()
geo_df.to_file('abag_2010_ccag_dist')

del vta_final['geometry']
vta_final.to_csv('ccag_final_abag_2010.csv',index=False)
vta_final.to_excel('ccag_final_abag_2010.xlsx',index=False)

In [ ]:
# Generate VTA Calculations

rp_calc = prep_data(abag, vta_taz)
vta_final = landuse_calcs(rp_calc)

geometry = vta_final['geometry']
crs = mtc_taz.crs
geo_df = gpd.GeoDataFrame(vta_final, crs=crs, geometry=geometry)


geo_df.head()
geo_df.to_file('abag_2010_vta_dist')

del vta_final['geometry']
vta_final.to_csv('vta_final_abag_2010.csv',index=False)
vta_final.to_excel('vta_final_abag_2010.xlsx',index=False)